# The simulation of Bessel-Gauss (BG) beams

Here we show how to use a customised input laser field instead of the default Gaussian pulse. This is reached by replacing the Gaussian pulse in the pre-processed CUPRAD input. This notebooks aims for specifying the field, the TDSE and harmonic part is not treated there. The input file is fully compatible and there is no difficulty [to add these steps](../gas_cell/prepare_cell.ipynb).

The approach to implement the BG beams is the following:
1) Set a *reference Gaussian pulse* and other input parameters.
2) Call the pre-processor locally from this notebook.
3) Replace the Gaussian pulse by our customised field in the pre-processed hdf5 archive.
4) The modified pre-processed hdf5 archive is then used in the standard operation of the code.

Note that the computational grids and other CUPRAD internal subroutines are set based on the input Gaussian pulse. Therefore, the parameters of *the reference Gaussian pulse* must symphonise with the replacing field.

## Load libraries

In [ ]:
import numpy as np
import itertools
from scipy import integrate
import matplotlib.pyplot as plt
import os
import shutil
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
import HHG
from IPython.display import display, Markdown


# %matplotlib inline

The path where the input parameters are written:

In [ ]:
outputs_path = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'Bessel','inputs')

## Physical parameters

Here we define the parameters of the *medium*, *Bessel-Gauss beams* and [*reference Gaussian beam*](../gas_cell/prepare_cell.ipynb).

We start with the medium.

In [ ]:
# gas specifiers
gas              = 'Ar'
medium_length    = 12.0e-2 # [m] 
ionisation_model = 'PPT'

medium_pressure = 25e-3 # [mbar]

Then we proceed to the definitions of the Bessel-Gauss beams: $$\mathcal{E}_{\mathrm{BG}}(\rho) = \mathcal{E}_0 \mathrm{e}^{-\left(\frac{\rho}{w_0}\right)^2}J_0 \left(\frac{2 r \rho}{w_0}\right)\,,$$
where we can identify the Bessel waist $w_{\mathrm{B}} = w_0/(2r)$, where $r=\theta_{\mathrm{B}} / \theta_{\mathrm{G}}$ is the ratio (denoted below as the `Bessel_ratios`) of the angular aperture of the cone $\theta_{\mathrm{B}}$ related to the Bessel part of the beam and the angular spread $\theta_{\mathrm{G}}$ of the Gaussian part of the beam, see Eq. (3.3) [here](https://doi.org/10.1016/0030-4018(87)90276-8) for more details.

We note that the solution transforms into a regular Gaussian beam for $r=0$. We use this case as the *reference Gaussian beam*. Generally, the profile is close to Gaussian for $r<1$ and the [diffraction-free Bessel properties](https://doi.org/10.1103/PhysRevLett.58.1499) are retrieved for $r>1$. Therefore, we fully specify the BG beams by $\mathcal{E}_0$, $w_0$ and $r$.

The final field further considers a Gaussian temporal profile:
$$\mathcal{E}(\rho,t) = \mathcal{E}_{\mathrm{BG}}(\rho) \mathrm{e}^{-\left(\frac{t}{\tau}\right)^2} \cos(\omega_0 t)\,,$$
which reads in Python:

In [ ]:
from scipy.special import j0 as J0

def BG0(r_,t_,E0_,w0_,ratio_,lambd_,t_e_):
    return E0_*np.exp(-(r_/w0_)**2) * np.exp(-(t_/t_e_)**2) *\
           J0(2. * ratio_ * r_ / w0_) *\
           np.cos(mn.ConvertPhoton(lambd_,'lambdaSI','omegaSI')*t_)

We place the entry of the medium at the focus of the beam, i.e., at the position with 0 curvature of the driving beam.

To compare BG beams with different $r$, we require them to share the waist $w_{\text{tot}}$ that we define via $\mathcal{E}_{\mathrm{BG}}(w_{\text{tot}})=1/\mathrm{e}$ (we define $w_{\text{tot}}=$`BG_beamwaist`). The unknown variable is now 'the Gaussian waist' $w_0$.

Here we set $w_{\text{tot}}$, the ratios $r$ and use a numerical routine to find corresponding $w_0$. (Note that $w_{\text{tot}}=w_0$ for $r=0$.)

In [ ]:
BG_beamwaist =   50e-6 # [m]
Bessel_ratios    = [0,   2.,   5.]
reference_Gaussian_focus = 0. # [m] there is no point to set it differently. (It defines but the reference field, which will be rewriten.)

# numerically find corresponding w0's
from scipy import optimize
w_Gauss = [(optimize.minimize(lambda x: (BG0(BG_beamwaist,0.,1.,x,ratio,1.,1.)-1./np.e)**2,
                                        (1.+ratio)*BG_beamwaist).x)[0] for ratio in Bessel_ratios]

Finally, we need to set the remaining parameters of the laser pulse: its focus intensity, duration and wavelength. We consider more intensities relative to the critical above-threshold-ionising intensity $I_{\text{ATI}}$. We use a rough estimate for this intensity considering a purely Coulombic potential suppresed by the external field to the ground-state energy level (the value of the electric field $\mathcal{E}$ for which the maximum of the potential $-x/(4\pi\varepsilon_0) - \mathcal{E}x = I_{p}$).

In [ ]:
laser_pulse_duration = 30e-15 # [s] (defined via 1/e in the electric field amplitude)
laser_wavelength = 800e-9 # [m]


Intensity_ratios = [0.1, 0.4, 0.7]
reference_Gaussian_focus_intensity =  HHG.Critical_ATI_intensity_rough(HHG.Ip_list[gas])*units.INTENSITYau  # [W/m2]

# Numerical parameters

Here we define the numerical parameters. This release of the code leaves the responsibility of choosing proper parameters to users, except the implementatation of adaptive steps in $z$. (See the [first tutorial](../gas_cell/prepare_cell.ipynb) for more details.)

In [ ]:
number_of_points_in_r      = 2*4096
number_of_points_in_t      = 512 # 1024

operators_t                = 2
first_delta_z              = 0.01 # [mm]
phase_threshold_for_decreasing_delta_z = 0.002	# [rad]

length_of_window_for_r_normalized_to_beamwaist = 16.   # [-]
length_of_window_for_t_normalized_to_pulse_duration = 6. # [-]

number_of_absorber_points_in_time = 16  # [-]

physical_output_distance_for_plasma_and_Efield = 0.0015   # [m]

output_distance_in_z_steps_for_fluence_and_power   = 100  # [-]

radius_for_diagnostics = 0.1 # [cm]

run_time_in_hours = 23.0 # [h] 

In [ ]:
## Code to generate the following text ##
reference_Gaussian_waist = BG_beamwaist
zR = (np.pi*reference_Gaussian_waist**2)/laser_wavelength
dr_CUPRAD = length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)/number_of_points_in_r
zR_list = [(np.pi*foo**2)/laser_wavelength for foo in w_Gauss]
display(Markdown(rf"""### Properties of the chosen discretisation & beam heuristics
* The chosen discretisation in time gives ~ {
            number_of_points_in_t/(
            laser_pulse_duration*length_of_window_for_t_normalized_to_pulse_duration/mn.ConvertPhoton(laser_wavelength,'lambdaSI','T0SI')
            )
    :.0f}
points per one laser period.
* The stepsize in the radial discretisation is ~ ${
      1e6*dr_CUPRAD
      :.2f}
~\mu {{\mathrm{{m}}}}$.
* The size of the radial computational box is ~ ${
      1e6*length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist
      :.2f}
~\mu {{\mathrm{{m}}}}$. The maximal radius of the reference Gaussian beam is ~ ${
      1e6*np.max([
            reference_Gaussian_waist*np.sqrt(1+((medium_length-reference_Gaussian_focus)/zR)**2),
            reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)
            ])
      :.2f}
~\mu {{\mathrm{{m}}}}$.$^\dagger$
* The Rayleigh length for the purely Gaussian beam is ${
      1e3*zR
      :.2f}
~{{\mathrm{{mm}}}}$ (the length of the cell is ${
      1e3*medium_length
      :.2f}
~{{\mathrm{{mm}}}}$).
* There will be ~ {
    medium_length/physical_output_distance_for_plasma_and_Efield
    :.0f} output planes.
* Propagation distances relative to the Rayleigh lengths ($L/z_R$): """+ ", ".join(["{:.1f}".format(medium_length/foo) for foo in zR_list])+"."+
rf"""

$^\dagger$ This is given at the $z$-edges of the computational box.
"""))

### Bessel-Gauss beam on our grids
Let's plot the initial intensity profiles of the beams on our computational grids. (We normalise the intensity as this scaling will be treated in the final plots.)

In [ ]:
### Code to generate the following figure ###

# Original Gauss: reconstruct grids etc.
w0    = reference_Gaussian_waist
rmax  = length_of_window_for_r_normalized_to_beamwaist * w0
rgrid = np.linspace(0.,rmax,number_of_points_in_r)

T_computational_window = laser_pulse_duration * length_of_window_for_t_normalized_to_pulse_duration
tgrid = np.linspace(-0.5*T_computational_window, 0.5*T_computational_window, number_of_points_in_t)
tv, rv = np.meshgrid(tgrid,rgrid) # vector grids

# plot the intensity profiles for the different ratios

fig, ax = plt.subplots()
for args, kwargs in zip([[1e6*rgrid,BG0(rgrid,0,1.,foo,ratio,laser_wavelength,1.)**2] for ratio, foo in zip(Bessel_ratios,w_Gauss)],
                        [{'label' : f'$r={ratio:.1f}$'} for ratio in Bessel_ratios]):    
    ax.plot(*args,**kwargs)

ax.legend()

ax.set_xlabel(r'$\rho~[\mu\mathrm{m}]$')
ax.set_ylabel(r'$I$ [-]')
ax.set_title(r'The tranversal profiles at the peak of the pulse')
plt.show()  

## Prepare the input file

Last, we create the HDF5 file containing all the input parameters. The procedure is:

1) Provide several dictionaries (for different modules) to translate the local variables used in this jupyter notebook to the nomenclature used in the code.
2) Create the archive with the reference Gaussian beam.
3) Run the pre-processor.
4) The reference archive is copied and the input field is replaced by the Bessel-Gauss profiles.

The path was set at the beginning pointing to the working directory.


In [ ]:
## Prepare dictionaries between hdf5-inputs and this jupyter notebook

global_input_names_to_jupyter_variables = {
    'gas_preset'                                : (np.bytes_(gas),                       '[-]'   ),
    'medium_pressure_in_bar'                    : (medium_pressure,                      '[bar]' )
}


CUPRAD_names_to_jupyter_variables = {
    # laser parameters
    'laser_wavelength'                          : (1e2*laser_wavelength,                  '[cm]'  ),
    'laser_pulse_duration_in_1_e_Efield'        : (1e15*laser_pulse_duration,             '[fs]' ),
    'laser_focus_intensity_Gaussian'            : (reference_Gaussian_focus_intensity,    '[W/m2]'  ),
    'laser_focus_beamwaist_Gaussian'            : (reference_Gaussian_waist,              '[m]'  ),
    'laser_focus_position_Gaussian'             : (reference_Gaussian_focus,              '[m]'  ),

    # medium parameters
    'medium_physical_distance_of_propagation'   : (medium_length,                         '[m]'   ),

    # ionisation
    'ionization_model'                          : (np.bytes_(ionisation_model),          '[-]'  ),

    # numerics
    'numerics_number_of_points_in_r'            : (number_of_points_in_r,                 '[-]'  ),
    'numerics_number_of_points_in_t'            : (number_of_points_in_t,                 '[-]'  ),
    'numerics_operators_t_t-1'                  : (operators_t,                           '[-]'  ),
    'numerics_physical_first_stepwidth'         : (first_delta_z,                         '[mm]' ),
    'numerics_phase_threshold_for_decreasing_delta_z' : 
        (phase_threshold_for_decreasing_delta_z,                '[rad]' ),
    'numerics_length_of_window_for_r_normalized_to_beamwaist':
        (length_of_window_for_r_normalized_to_beamwaist,        '[-]'   ),
    'numerics_length_of_window_for_t_normalized_to_pulse_duration' :
        (length_of_window_for_t_normalized_to_pulse_duration,   '[-]'   ),
    'numerics_number_of_absorber_points_in_time':
        (number_of_absorber_points_in_time ,                    '[-]'   ),
    'numerics_physical_output_distance_for_plasma_and_Efield' :
        (physical_output_distance_for_plasma_and_Efield,        '[m]'   ),
    'numerics_output_distance_in_z-steps_for_fluence_and_power' :
        (output_distance_in_z_steps_for_fluence_and_power,      '[-]'   ),
    'numerics_radius_for_diagnostics'           : (radius_for_diagnostics,                '[cm]' ),
    'numerics_run_time_in_hours'                : (run_time_in_hours,                     '[s]'  )
}

In [ ]:
## Create the hdf5-archive
h5filename = 'results_Bessel.h5'

if os.path.exists(outputs_path): shutil.rmtree(outputs_path)  # clean the input directory if it existed
os.makedirs(outputs_path)
h5filepath = os.path.join(outputs_path,h5filename)

from inputs_transformer import add_variables2hdf5
with h5py.File(h5filepath,'w') as f: 

    add_variables2hdf5(f,
                    global_input_names_to_jupyter_variables,
                    CUPRAD_names_to_jupyter_variables,
                    None,
                    None,
                    None,
                    None)

In [ ]:
## run pre-processor
import subprocess

os.chdir(outputs_path)

inputs = f"{h5filename}\n0\n0\n0\n"

for foo in ['MERGE_RAD.LOG', 'STOP', 'listing', 'PROP_RAD.LOG']:
      try: os.remove(foo)
      except: pass

log = subprocess.run(
      [os.environ['CUPRAD_BUILD']+'/make_start.e'],  # command to run the script
      input=inputs,                # provide input here
      text=True,                   # treat input/output as text (str)
      capture_output=True          # capture stdout and stderr
)

We copy the input files with the list of various Bessel-Gauss beams.

Note that the code works with the complexified field in the time domain, it is defined as $$\mathcal{E}_{\text{cmplx}}(t) = \mathscr{F}[\theta (\omega)\hat{\mathcal{E}}(\omega)](t) \,,$$ where $\theta (\omega)$ is the Heaviside function and $\hat{\mathcal{E}}(\omega)$ is the Fourier transform of $\mathcal{E}(t)$.

We also plot the input fields and the peak intensity profile for each input. We can choose several units for plotting: $\mathrm{W/cm}^2$, relative to $I_{\text{ATI}}$, or relative to the harmonic cutoff.

In [ ]:
# the filename convention
new_h5file_stub = 'results_Bessel_'

# The units for plotting
intensity_scale = 'W/cm2'      # {W/cm2, cutoff, critical}   

# mask to plot the intensity in the chosen units
intensity_mask = lambda x: (1e-4*x
                            if (intensity_scale == 'W/cm2')
                            else 
                            HHG.ComputeCutoff(
                             x/units.INTENSITYau,
                             mn.ConvertPhoton(laser_wavelength,'lambdaSI','omegaau'),
                             HHG.Ip_list[gas]
                           )[1]
                           if (intensity_scale == 'cutoff')
                           else
                           1e2*x/(HHG.Critical_ATI_intensity_rough(HHG.Ip_list[gas])*units.INTENSITYau)
                           )


# Prepare figures to show the input fields
Nsim = len(Bessel_ratios) * len(Intensity_ratios)
Nrows = Nsim//3 if (Nsim%3==0) else (Nsim//3)+1

fig1, axes1 = plt.subplots(Nrows, 3, figsize=(17, Nrows*4))  # Create a 2x2 grid of subplots
axes1 = axes1.flatten()

fig2, axes2 = plt.subplots(Nrows, 3, figsize=(17, Nrows*4))  # Create a 2x2 grid of subplots
axes2 = axes2.flatten()

for k1, (k2, k3) in enumerate(itertools.product(range(len(Bessel_ratios)),range(len(Intensity_ratios)))):

    E0 = np.sqrt(Intensity_ratios[k3] * reference_Gaussian_focus_intensity/units.INTENSITYau)*units.EFIELDau
    new_field = BG0(rv,tv,E0,w_Gauss[k2],Bessel_ratios[k2],laser_wavelength,laser_pulse_duration)

    axes2[k1].plot(1e6*rgrid,intensity_mask(mn.FieldToIntensitySI(BG0(rgrid,0,E0,w_Gauss[k2],Bessel_ratios[k2],laser_wavelength,1.))))
    axes2[k1].set_xlabel(r'$\rho~[\mu m]$')
    axes2[k1].set_ylabel(r'$I$ ['+('$\mathrm{W/cm^2}$' if (intensity_scale == 'W/cm2') else
                                   'cutoff'   if (intensity_scale == 'cutoff') else
                                   r'% of $I_{\text{ATI}}$')+']')
    axes2[k1].set_title('sim '+str(k1+1)+rf', peak intensity') 

    # Complexification
    Nr, Nt = np.shape(new_field)
    new_field_cmplx = np.empty((Nr, Nt),dtype=np.cdouble)
    for k4 in range(Nr): 
        new_field_cmplx[k4,:] = np.conj(mn.complexify_fft(new_field[k4,:]))


    pc1 = axes1[k1].pcolormesh(1e15*tgrid, 1e6*rgrid, 1e-9*new_field_cmplx.real, shading='auto', cmap='seismic')    
    cbar = fig1.colorbar(pc1, ax=axes1[k1])
    cbar.set_label('$\mathcal{E}$ [GV/m]', rotation=90)
    axes1[k1].set_xlabel(r'$t~[fs]$')
    axes1[k1].set_ylabel(r'$\rho~[\mu m]$')
    axes1[k1].set_title('sim '+str(k1+1)+rf', $r={Bessel_ratios[k2]}$ [-], $\frac{{I_0}}{{I_{{\text{{crit.}}}}}}={Intensity_ratios[k3]}$ [-]')


    new_directory = new_h5file_stub+str(k1+1)
    if os.path.exists(new_directory): shutil.rmtree(new_directory)
    os.makedirs(new_directory)
    new_h5file = os.path.join(new_directory,new_h5file_stub+str(k1+1)+'.h5')
    new_msg_file = os.path.join(new_directory,'msg.tmp')

    shutil.copy2(h5filename, new_h5file)
    for foo in ['MERGE_RAD.LOG', 'STOP', 'listing', 'PROP_RAD.LOG']:
      new_file = os.path.join(new_directory,foo)
      try: shutil.copy2(foo,new_file)
      except: pass
 
    with h5py.File(new_h5file,'r+') as f, open(new_msg_file,'w') as msg:
        f[MMA.paths['CUPRAD_pre-processed']+'/startfield_r'][:]=new_field_cmplx.real
        f[MMA.paths['CUPRAD_pre-processed']+'/startfield_i'][:]=new_field_cmplx.imag
        msg.write(new_h5file_stub+str(k1+1)+'.h5')


for k1 in range(Nsim,3*Nrows):
  axes1[k1].axis('off')
  axes2[k1].axis('off')
fig1.tight_layout() 
fig2.tight_layout() 
plt.show()